## Easy Experiment .ipynb File

### <font color='red'><b> 🐬 기억합시다 :] 실험마다 인덱스 붙여서 파일 새로만들기~! <b></font>

### unseen data로 validation을 측정하는 실험

## inference

## inference config

In [34]:
import torch
import torch.nn as nn
from torch.optim import *
from torch.optim.lr_scheduler import *
import torch.nn.functional as F

import random
import time
from datetime import timedelta

import numpy as np

import albumentations as A
from albumentations.pytorch import ToTensorV2
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from pytorch_model_summary import summary

from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score

import multiprocessing
import timm

from PIL import Image

import os
import json

import pandas as pd
import matplotlib.pyplot as plt
from torchvision.utils import make_grid

from sklearn.model_selection import train_test_split

In [35]:
model_name = 'resnet34'
save_file_path = f'./checkpoints/soup/first_soup.pth'
test_dir = './dataset/eval/images/'
test_csv_path = './dataset/eval/info.csv'

test_b_size = 500
num_labels = 9
sumbission_csv_name = 'first_soup'

In [36]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

## checkpoint model load

In [37]:
model = timm.create_model(model_name=model_name, pretrained=False, num_classes=num_labels)
model.load_state_dict(torch.load(save_file_path))
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
      (act1): ReLU(inplace=True)
      (aa): Identity()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act2): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, m

## test dataset

In [38]:
class CustomTestDataset(Dataset):
    def __init__(self, img_paths:list, transforms=None):
        self.img_paths = img_paths
        self.transforms = transforms
        assert self.transforms is not None, 'you must use transforms in Testset'

    def __getitem__(self, index):
        img = Image.open(self.img_paths[index])
        img = np.array(img)
        img = self.transforms(image=img)["image"]
        return img

    def __len__(self):
        return len(self.img_paths)

In [39]:
test_transforms = A.Compose([
        A.CenterCrop(height=450,width=250),
        A.Resize(224, 224),
        A.Normalize(mean=(0.560,0.524,0.501), std=(0.233,0.243,0.246)),
        ToTensorV2()
    ])

In [40]:
info = pd.read_csv(test_csv_path)
test_imgs_path = [os.path.join(test_dir, img_id) for img_id in info.ImageID]

In [41]:
test_dset = CustomTestDataset(test_imgs_path, test_transforms)

In [42]:
test_loader = DataLoader(
        test_dset,
        batch_size=test_b_size,
        num_workers=multiprocessing.cpu_count() // 2,
        shuffle=False,
        pin_memory=use_cuda,
        drop_last=False,
    )

In [43]:
mask_pred_cls = []
gender_pred_cls = []
age_pred_cls = []

with torch.no_grad():
        for idx, images in enumerate(test_loader):
            images = images.to(device)
            output = model(images)
            mask_logits, gender_logits, age_logits, _ = torch.split(output, [3,2,3,1], dim=1)
            mask_pred = mask_logits.argmax(dim=-1)
            gender_pred = gender_logits.argmax(dim=-1)
            age_pred = age_logits.argmax(dim=-1)

            mask_pred_cls += mask_pred.cpu().numpy().tolist()
            gender_pred_cls += gender_pred.cpu().numpy().tolist()
            age_pred_cls += age_pred.cpu().numpy().tolist()

In [44]:
## for True Label
def get_label(mask_state, gender, age_cls):
    label = 0
    if mask_state == 0 and gender == 0 and age_cls == 0: # 0:
        label = 0
    elif mask_state == 0 and gender == 0 and age_cls == 1: # 1
        label = 1
    elif mask_state == 0 and gender == 0 and age_cls == 2: # 2
        label = 2
    elif mask_state == 0 and gender == 1 and age_cls == 0: # 3
        label = 3
    elif mask_state == 0 and gender == 1 and age_cls == 1: # 4
        label = 4
    elif mask_state == 0 and gender == 1 and age_cls == 2: # 5
        label = 5
    elif mask_state == 1 and gender == 0 and age_cls == 0: # 6
        label = 6
    elif mask_state == 1 and gender == 0 and age_cls == 1: # 7
        label = 7
    elif mask_state == 1 and gender == 0 and age_cls == 2: # 8
        label = 8
    elif mask_state == 1 and gender == 1 and age_cls == 0: # 9
        label = 9
    elif mask_state == 1 and gender == 1 and age_cls == 1: # 10
        label = 10
    elif mask_state == 1 and gender == 1 and age_cls == 2: # 11
        label = 11
    elif mask_state == 2 and gender == 0 and age_cls == 0: # 12
        label = 12
    elif mask_state == 2 and gender == 0 and age_cls == 1: # 13
        label = 13
    elif mask_state == 2 and gender == 0 and age_cls == 2: # 14
        label = 14
    elif mask_state == 2 and gender == 1 and age_cls == 0: # 15
        label = 15
    elif mask_state == 2 and gender == 1 and age_cls == 1: # 16
        label = 16
    elif mask_state == 2 and gender == 1 and age_cls == 2: # 17
        label = 17
    else:
        raise ValueError
    return label

pred_cls = [get_label(m,g,a) for (m,g,a) in zip(mask_pred_cls, gender_pred_cls, age_pred_cls)]

In [45]:
save_dir = './checkpoints/'
info['ans'] = pred_cls
save_path = os.path.join(save_dir, f'{sumbission_csv_name}.csv')
info.to_csv(save_path, index=False)
print(f"Inference Done! Inference result saved at {save_path}")

Inference Done! Inference result saved at ./checkpoints/first_soup.csv
